In [3]:
# 导入库
import cv2
import time
from math import*
import numpy as np
import keyboard

KNN_subtractor = cv2.createBackgroundSubtractorKNN(detectShadows = True)

MOG2_subtractor = cv2.createBackgroundSubtractorMOG2(detectShadows = True)

bg_subtractor=MOG2_subtractor

mediaName = ".\WIN_20240822_15_44_20_Pro.mp4"
camera = cv2.VideoCapture(mediaName)
waitMillSec = 17

recFrame = 0
# tailColor = ()

_, fristFrame = camera.read()
availableMask = np.ones(fristFrame.shape, dtype= np.uint8)
while True:
    gROI = cv2.selectROI("ROI frame", fristFrame * availableMask, False)

    availableMask[gROI[1]:(gROI[1] + gROI[3]), gROI[0]:(gROI[0] + gROI[2])] = 0

    if gROI == (0,0,0,0):
        cv2.destroyAllWindows()
        break



In [8]:
cv2.imshow("temp", availableMask * 255)
cv2.waitKey()
cv2.destroyAllWindows()

In [12]:
while True:
    ret, frame = camera.read()
    frame = frame * availableMask

    # 每一帧既用于计算前景掩码，也用于更新背景。
    foreground_mask = bg_subtractor.apply(frame)

    # 如果大于240像素，则阈值设为255，如果小于则设为0    # 创建二值图像，它只包含白色和黑色像素
    ret , threshold = cv2.threshold(foreground_mask.copy(), 120, 255, cv2.THRESH_BINARY)

    # 膨胀扩展或加厚图像中的兴趣区域。
    threshold = cv2.medianBlur(threshold, 7)
    dilated = cv2.dilate(threshold, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3,3)), iterations = 4)

    # 查找轮廓
    contours, hier = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # 检查每个轮廓是否超过某个值，如果超过则绘制边界框
    for contour in contours:
        tempArea = cv2.contourArea(contour)
        if tempArea > 5000 and tempArea < 15000:
            (x,y,w,h) = cv2.boundingRect(contour)
            if recFrame % 10 == 0:
                cv2.imwrite("Output" + str(time.strftime("%m%d%H%M%S", time.gmtime()))+"/"+ mediaName +"ROI"+str(int(recFrame / 10))+".jpg", frame[y:y+h, x:x+w])
            cv2.rectangle(frame, (x,y), (x+w, y+h), (255,255,0), 2)
            print(tempArea)
            
            recFrame+=1
            break

    else:
        continue
    

    cv2.imshow("Subtractor", foreground_mask)
    cv2.imshow("threshold", threshold)
    cv2.imshow("detection", frame)
    key = cv2.waitKey(waitMillSec) & 0xff
    if key != 255:
        if key == 27:
            break
        elif key == 32:
            cv2.waitKey()
        # elif key == 
        # elif key == 0:
        #     waitMillSec = 2
        #     pass
        # else:
        #     waitMillSec = 17
        # print("key press")
    
    # elif key == 120 or key == 88:
    #     gROI = cv2.selectROI("ROI frame", frame, False)
    #     ROI = frame[gROI[1]:(gROI[1] + gROI[3]), gROI[0]:(gROI[0] + gROI[2])]

    #     cv2.waitKey()
camera.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 